# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

## Imports

In [1]:
import pandas as pd
import numpy as np
import math
import json
import datetime
%matplotlib inline

# read in the json files
portfolio = pd.read_json('./data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('./data/profile.json', orient='records', lines=True)
transcript = pd.read_json('./data/transcript.json', orient='records', lines=True)

## Exploring data

In [2]:
portfolio

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7
5,3,"[web, email, mobile, social]",7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2
6,2,"[web, email, mobile, social]",10,10,discount,fafdcd668e3743c1bb461111dcafc2a4
7,0,"[email, mobile, social]",0,3,informational,5a8bc65990b245e5a138643cd4eb9837
8,5,"[web, email, mobile, social]",5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d
9,2,"[web, email, mobile]",10,7,discount,2906b810c7d4411798c6938adc9daaa5


**Checking for missing values**

In [3]:
portfolio.isnull().sum()

reward        0
channels      0
difficulty    0
duration      0
offer_type    0
id            0
dtype: int64

**Portfolio has 10 instances with no missing data!**

**Displaying useful info about the portfolio dataset**

In [4]:
portfolio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
reward        10 non-null int64
channels      10 non-null object
difficulty    10 non-null int64
duration      10 non-null int64
offer_type    10 non-null object
id            10 non-null object
dtypes: int64(3), object(3)
memory usage: 608.0+ bytes


In [5]:
portfolio.describe()

,reward,difficulty,duration
count,10.000000,10.000000,10.000000
mean,4.200000,7.700000,6.500000
std,3.583915,5.831905,2.321398
min,0.000000,0.000000,3.000000
25%,2.000000,5.000000,5.000000
50%,4.000000,8.500000,7.000000
75%,5.000000,10.000000,7.000000
max,10.000000,20.000000,10.000000


**Observations:**
- Reward
    - Average offer reward is approximately 4.
    - Maximum reward was 10
- Difficulty
    - Average offer difficulty is approximately 8.
    - Maximum difficulty was 20
- Duration
    - On average, an offer lasts for 6.5 days.
    - The longest an offer last was for 10 day while the shortest lasted for only 3 days.

**Checking offers' categories**

In [6]:
portfolio.groupby('offer_type').id.count()

offer_type
bogo             4
discount         4
informational    2
Name: id, dtype: int64

**There are 4 BOGO offers, 4 discounts, and 2 informational offers.**

In [7]:
def split_channels(portfolio):
    
    # get channels values
    channels_set = set()
    for i in portfolio.channels:
        channels_set.update(set(i))

    # split the channel into different attribute
    for i in channels_set:
        portfolio[i] = portfolio.channels.apply(lambda x: i in x).map({True:1, False: 0})
    
    # drop the channels column as we don't need it anymore
    portfolio.drop('channels', axis=1, inplace=True)
    
    return portfolio   # return the dataframe

In [8]:
portfolio = split_channels(portfolio)
portfolio

,reward,difficulty,duration,offer_type,id,web,mobile,social,email
0,10,10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd,0,1,1,1
1,10,10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0,1,1,1,1
2,0,0,4,informational,3f207df678b143eea3cee63160fa8bed,1,1,0,1
3,5,5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,1,0,1
4,5,20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7,1,0,0,1
5,3,7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2,1,1,1,1
6,2,10,10,discount,fafdcd668e3743c1bb461111dcafc2a4,1,1,1,1
7,0,0,3,informational,5a8bc65990b245e5a138643cd4eb9837,0,1,1,1
8,5,5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d,1,1,1,1
9,2,10,7,discount,2906b810c7d4411798c6938adc9daaa5,1,1,0,1


In [9]:
# encoding offer ids for easier readings
offers = dict(zip(portfolio['id'], portfolio.index.values.tolist()))
offers

{'ae264e3637204a6fb9bb56bc8210ddfd': 0,
 '4d5c57ea9a6940dd891ad53e9dbe8da0': 1,
 '3f207df678b143eea3cee63160fa8bed': 2,
 '9b98b8c7a33c4b65b9aebfe6a799e6d9': 3,
 '0b1e1539f2cc45b7b9fa7c272da2e1d7': 4,
 '2298d6c36e964ae4a3e7e9706d1fb8c2': 5,
 'fafdcd668e3743c1bb461111dcafc2a4': 6,
 '5a8bc65990b245e5a138643cd4eb9837': 7,
 'f19421c1d4aa40978ebb69ca19b0e20d': 8,
 '2906b810c7d4411798c6938adc9daaa5': 9}

In [10]:
# encoding offers ids
# adding 1 to make them from 1:10 instead of 0:9 (simpler for human reading)
portfolio['id'] = portfolio['id'].apply(lambda x: offers[x]+1)
portfolio

,reward,difficulty,duration,offer_type,id,web,mobile,social,email
0,10,10,7,bogo,1,0,1,1,1
1,10,10,5,bogo,2,1,1,1,1
2,0,0,4,informational,3,1,1,0,1
3,5,5,7,bogo,4,1,1,0,1
4,5,20,10,discount,5,1,0,0,1
5,3,7,7,discount,6,1,1,1,1
6,2,10,10,discount,7,1,1,1,1
7,0,0,3,informational,8,0,1,1,1
8,5,5,5,bogo,9,1,1,1,1
9,2,10,7,discount,10,1,1,0,1


## Exploring profile data

In [11]:
profile.head()

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


**Checking for missing values**

In [12]:
profile.isnull().sum()

gender              2175
age                    0
id                     0
became_member_on       0
income              2175
dtype: int64

In [13]:
profile[profile.isnull()]

,gender,age,id,became_member_on,income
0,None,NaN,None,NaN,NaN
1,None,NaN,None,NaN,NaN
2,None,NaN,None,NaN,NaN
3,None,NaN,None,NaN,NaN
4,None,NaN,None,NaN,NaN
...,...,...,...,...,...
16995,NaN,NaN,NaN,NaN,NaN
16996,NaN,NaN,NaN,NaN,NaN
16997,NaN,NaN,NaN,NaN,NaN
16998,NaN,NaN,NaN,NaN,NaN


**Let's drop those rows, as they have no information and will not be useful to us.**

In [14]:
profile.dropna(inplace=True)

In [15]:
profile.isnull().sum()

gender              0
age                 0
id                  0
became_member_on    0
income              0
dtype: int64

**Displaying useful info about the data**

In [16]:
profile.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14825 entries, 1 to 16999
Data columns (total 5 columns):
gender              14825 non-null object
age                 14825 non-null int64
id                  14825 non-null object
became_member_on    14825 non-null int64
income              14825 non-null float64
dtypes: float64(1), int64(2), object(2)
memory usage: 694.9+ KB


**Displaying useful info about data**

In [17]:
print('Age Statistics:')
profile.groupby('gender')['age'].describe()

Age Statistics:


,count,mean,std,min,25%,50%,75%,max
gender,,,,,,,,
F,6129.0,57.544950,16.883315,18.0,48.0,58.0,69.0,101.0
M,8484.0,52.116690,17.413894,18.0,39.0,53.0,65.0,100.0
O,212.0,54.400943,16.244710,20.0,43.0,55.0,66.0,100.0


In [18]:
print('Income Statistics:')
profile.groupby(['gender'])['income'].describe()

Income Statistics:


,count,mean,std,min,25%,50%,75%,max
gender,,,,,,,,
F,6129.0,71306.412139,22338.353773,30000.0,54000.0,71000.0,88000.0,120000.0
M,8484.0,61194.601603,20069.517615,30000.0,45000.0,59000.0,73000.0,120000.0
O,212.0,63287.735849,18938.594726,30000.0,51000.0,62000.0,79250.0,100000.0


**Observations:**
- Age
    - Average age of female users is approximately 57.5 while the average for males is 52, other types' average is 54.
- Income
    - Females have the highest average income, followed by Males, then others.

**Extracting joined_in_year column** Which has the year each user joined in

In [40]:
def getMonth(x):
    '''
    Inputs:
        x: date in "%Y%m%d" format
    
    Returns:
        year
    '''
    
    return datetime.datetime.strptime(x, "%Y%m%d").year

In [42]:
profile['joined_in_year'] = profile['became_member_on'].apply(lambda x: getMonth(str(x)))

In [43]:
profile.head()

,gender,age,id,became_member_on,income,joined_in_year
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0,2017
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0,2017
5,M,68,e2127556f4f64592b11af22de27a7932,20180426,70000.0,2018
8,M,65,389bc3fa690240e798340f5a15918d5c,20180209,53000.0,2018
12,M,58,2eeac8d8feae4a8cad5a6af0499a211d,20171111,51000.0,2017


In [21]:
transcript.tail(20)

,person,event,value,time
306514,f1e4fd36e5a0446f83861308bddf6945,transaction,{'amount': 8.2},714
306515,0b64be3b241c4407a5c9a71781173829,transaction,{'amount': 2.6},714
306516,86d03d35d7e0434b935e7743e83be3a0,transaction,{'amount': 9.2},714
306517,3408fd05c781401f8442fb6dbaaea9c7,transaction,{'amount': 11.7},714
306518,1593d617fac246ef8e50dbb0ffd77f5f,transaction,{'amount': 40.67},714
306519,f1b31d07b5d84f69a2d5f1d07843989e,transaction,{'amount': 31.13},714
306520,2ce987015ec0404a97ba333e8e814090,transaction,{'amount': 1.6400000000000001},714
306521,2e33545f0a764d27b2ccff95fc8d72c4,transaction,{'amount': 17.35},714
306522,d1c4500ace2e45e9a45d3cd2fccac8d8,transaction,{'amount': 4.42},714
306523,b65affd9e07346a1906364a396950e3d,transaction,{'amount': 18.35},714


**Checking for missing values**

In [22]:
transcript.isna().sum()

person    0
event     0
value     0
time      0
dtype: int64

**extracting offer_id and other attributes from values column**

In [23]:
def extract_values(col):

    attrubutes = []        
    for idx, row in transcript.iterrows():
        for val in row['value']:
            if val in attrubutes:
                continue
            else:
                attrubutes.append(val)
    
    # we have 4 attributes: 'offer id', 'amount', 'offer_id', 'reward'
    # but 'offer id' and 'offer_id' are the same, we will use the latter one
    # let's encode them..
    
    # initializing columns
    transcript['offer_id'] = ''
    transcript['amount'] = 0
    transcript['reward'] = 0
    
    # iterating over transcript table
    for idx, row in transcript.iterrows():
        for val in row['value']:
            # check value column and update it, put each key in seperated column.
            if val == 'offer_id' or val == 'offer id':
                transcript.at[idx, 'offer_id']= row['value'][val]
            else:
                 transcript.at[idx, str(val)]= row['value'][val]
    
    # drop value column
    transcript.drop('value', axis=1, inplace=True)
                
    return transcript

In [24]:
transcript = extract_values(transcript)

In [25]:
transcript.head()

,person,event,time,offer_id,amount,reward
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0,0
1,a03223e636434f42ac4c3df47e8bac43,offer received,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,0
2,e2127556f4f64592b11af22de27a7932,offer received,0,2906b810c7d4411798c6938adc9daaa5,0,0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,0,fafdcd668e3743c1bb461111dcafc2a4,0,0
4,68617ca6246f4fbc85e91a2a49552598,offer received,0,4d5c57ea9a6940dd891ad53e9dbe8da0,0,0


In [26]:
transcript['event'].value_counts()

transaction        138953
offer received      76277
offer viewed        57725
offer completed     33579
Name: event, dtype: int64

In [27]:
users_who_received_offers = transcript[(transcript['event'] == 'offer received')]
users_who_received_offers

,person,event,time,offer_id,amount,reward
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0,0
1,a03223e636434f42ac4c3df47e8bac43,offer received,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,0
2,e2127556f4f64592b11af22de27a7932,offer received,0,2906b810c7d4411798c6938adc9daaa5,0,0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,0,fafdcd668e3743c1bb461111dcafc2a4,0,0
4,68617ca6246f4fbc85e91a2a49552598,offer received,0,4d5c57ea9a6940dd891ad53e9dbe8da0,0,0
...,...,...,...,...,...,...
257882,d087c473b4d247ccb0abfef59ba12b0e,offer received,576,ae264e3637204a6fb9bb56bc8210ddfd,0,0
257883,cb23b66c56f64b109d673d5e56574529,offer received,576,2906b810c7d4411798c6938adc9daaa5,0,0
257884,6d5f3a774f3d4714ab0c092238f3a1d7,offer received,576,2298d6c36e964ae4a3e7e9706d1fb8c2,0,0
257885,9dc1421481194dcd9400aec7c9ae6366,offer received,576,ae264e3637204a6fb9bb56bc8210ddfd,0,0


In [28]:
transcript[transcript['event'] == 'transaction']['person'].values.tolist()

['02c083884c7d45b39cc68e1314fec56c',
 '9fa9ae8f57894cc9a3b8a9bbe0fc1b2f',
 '54890f68699049c2a04d415abc25e717',
 'b2f1cd155b864803ad8334cdf13c4bd2',
 'fe97aa22dd3e48c8b143116a8403dd52',
 '629fc02d56414d91bca360decdfa9288',
 'bbeb54e861614fc7b22a8844f72dca6c',
 'a97e6f33219c432db82acfa0d19c602d',
 '676506bad68e4161b9bbaffeb039626b',
 '8f7dd3b2afe14c078eb4f6e6fe4ba97d',
 '4cbe33c601a5407f8202086565c55111',
 'b432b74402bb4981a4651c8df1670365',
 'a04fcfd571034456aaa6d56c0a3fd9b6',
 '227f2d69e46a4899b70d48182822cff6',
 'bb0f25e23a4c4de6a645527c275cd594',
 'c2c72ce6038644c797208046d1e3498a',
 '7ca349e55ff544c7a13addfdea2e2c06',
 'd72d201be5794279aa716d8ad82b8d90',
 'ad80753fc9e0485c9e6b1cc9478d827f',
 '73ffefd41e9a4ca3ab26b2b3697c6eb7',
 '3e621194f72e40d7a0b695ee9b7c38b7',
 '1f5c961416e64c5d88098b02b1bdf246',
 '3bcc51fdde354eb1949c813dbc905182',
 'ed46fca6de7042478b411690878dc069',
 'b860d355ef6e4c66b5d5a837c56ef32d',
 '4ad3748475204cf99571183f05b5e2f7',
 '99297ea01107436fa8c2e2bc86f55d89',
 

In [29]:
 users_who_received_offers['person'].values.tolist()

['78afa995795e4d85b5d9ceeca43f5fef',
 'a03223e636434f42ac4c3df47e8bac43',
 'e2127556f4f64592b11af22de27a7932',
 '8ec6ce2a7e7949b1bf142def7d0e0586',
 '68617ca6246f4fbc85e91a2a49552598',
 '389bc3fa690240e798340f5a15918d5c',
 'c4863c7985cf408faee930f111475da3',
 '2eeac8d8feae4a8cad5a6af0499a211d',
 'aa4862eba776480b8bb9c68455b8c2e1',
 '31dda685af34476cad5bc968bdb01c53',
 '744d603ef08c4f33af5a61c8c7628d1c',
 '3d02345581554e81b7b289ab5e288078',
 '4b0da7e80e5945209a1fdddfe813dbe0',
 'c27e0d6ab72c455a8bb66d980963de60',
 'd53717f5400c4e84affdaeda9dd926b3',
 'f806632c011441378d4646567f357a21',
 'd058f73bf8674a26a95227db098147b1',
 '65aba5c617294649aeb624da249e1ee5',
 'ebe7ef46ea6f4963a7dd49f501b26779',
 '1e9420836d554513ab90eba98552d0a9',
 '868317b9be554cb18e50bc68484749a2',
 'f082d80f0aac47a99173ba8ef8fc1909',
 '102e9454054946fda62242d2e176fdce',
 '4beeb3ed64dd4898b0edf2f6b67426d3',
 '9f30b375d7bd4c62a884ffe7034e09ee',
 '25c906289d154b66bf579693f89481c9',
 '6e014185620b49bd98749f728747572f',
 

In [30]:
#transcript.iloc[transcript[transcript['event'] == 'transaction']['person'] in users_who_received_offers['person'].values]

In [31]:
#users_who_completed_offers = transcript.iloc[transcript[(transcript['event'] == 'offer received')].index.values]
#users_who_completed_offers

In [32]:
transcript.groupby(['offer_id', 'event']).count()

person    time  amount  \
offer_id                         event                                     
                                 transaction      138953  138953  138953   
0b1e1539f2cc45b7b9fa7c272da2e1d7 offer completed    3420    3420    3420   
                                 offer received     7668    7668    7668   
                                 offer viewed       2663    2663    2663   
2298d6c36e964ae4a3e7e9706d1fb8c2 offer completed    5156    5156    5156   
                                 offer received     7646    7646    7646   
                                 offer viewed       7337    7337    7337   
2906b810c7d4411798c6938adc9daaa5 offer completed    4017    4017    4017   
                                 offer received     7632    7632    7632   
                                 offer viewed       4118    4118    4118   
3f207df678b143eea3cee63160fa8bed offer received     7617    7617    7617   
                                 offer viewed       4144    4144    4144   
4d5c57ea9a6940dd891ad53e9dbe8da0 offer completed    3331    3331    3331   
                                 offer received     7593    7593    7593   
                                 offer viewed       7298    7298    7298   
5a8bc65990b245e5a138643cd4eb9837 offer received     7618    7618    7618   
                                 offer viewed       6687    6687    6687   
9b98b8c7a33c4b65b9aebfe6a799e6d9 offer completed    4354    4354    4354   
                                 offer received     7677    7677    7677   
                                 offer viewed       4171    4171    4171   
ae264e3637204a6fb9bb56bc8210ddfd offer completed    3688    3688    3688   
                                 offer received     7658    7658    7658   
                                 offer viewed       6716    6716    6716   
f19421c1d4aa40978ebb69ca19b0e20d offer completed    4296    4296    4296   
                                 offer received     7571    7571    7571   
                                 offer viewed       7264    7264    7264   
fafdcd668e3743c1bb461111dcafc2a4 offer completed    5317    5317    5317   
                                 offer received     7597    7597    7597   
                                 offer viewed       7327    7327    7327   

                                                  reward  
offer_id                         event                    
                                 transaction      138953  
0b1e1539f2cc45b7b9fa7c272da2e1d7 offer completed    3420  
                                 offer received     7668  
                                 offer viewed       2663  
2298d6c36e964ae4a3e7e9706d1fb8c2 offer completed    5156  
                                 offer received     7646  
                                 offer viewed       7337  
2906b810c7d4411798c6938adc9daaa5 offer completed    4017  
                                 offer received     7632  
                                 offer viewed       4118  
3f207df678b143eea3cee63160fa8bed offer received     7617  
                                 offer viewed       4144  
4d5c57ea9a6940dd891ad53e9dbe8da0 offer completed    3331  
                                 offer received     7593  
                                 offer viewed       7298  
5a8bc65990b245e5a138643cd4eb9837 offer received     7618  
                                 offer viewed       6687  
9b98b8c7a33c4b65b9aebfe6a799e6d9 offer completed    4354  
                                 offer received     7677  
                                 offer viewed       4171  
ae264e3637204a6fb9bb56bc8210ddfd offer completed    3688  
                                 offer received     7658  
                                 offer viewed       6716  
f19421c1d4aa40978ebb69ca19b0e20d offer completed    4296  
                                 offer received     7571  
                                 offer viewed       7264  
fafdcd668e3743c1bb461111dcafc2a4 offer co

In [33]:
transcript.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306534 entries, 0 to 306533
Data columns (total 6 columns):
person      306534 non-null object
event       306534 non-null object
time        306534 non-null int64
offer_id    306534 non-null object
amount      306534 non-null int64
reward      306534 non-null int64
dtypes: int64(3), object(3)
memory usage: 14.0+ MB


**Displaying useful info about data**

In [34]:
print('Transcript Statistics:')
transcript.describe()

Transcript Statistics:


,time,amount,reward
count,306534.000000,306534.000000,306534.000000
mean,366.382940,5.570133,0.537219
std,200.326314,21.266669,1.805208
min,0.000000,0.000000,0.000000
25%,186.000000,0.000000,0.000000
50%,408.000000,0.000000,0.000000
75%,528.000000,7.000000,0.000000
max,714.000000,1062.000000,10.000000


**Observations:**
- Age
    - Average age of female users is approximately 57.5 while the average for males is 52, other types' average is 54.
- Income
    - Females have the highest average income.

**Checking for missing values**

In [35]:
profile.isnull().sum()

gender              0
age                 0
id                  0
became_member_on    0
income              0
joined_in_year      0
dtype: int64

In [36]:
profile[profile.isnull()]

,gender,age,id,became_member_on,income,joined_in_year
1,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN
12,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
16995,NaN,NaN,NaN,NaN,NaN,NaN
16996,NaN,NaN,NaN,NaN,NaN,NaN
16997,NaN,NaN,NaN,NaN,NaN,NaN
16998,NaN,NaN,NaN,NaN,NaN,NaN


**Let's drop those rows, as they have no information and will not be useful to us.**

In [37]:
profile.dropna(inplace=True)

In [38]:
profile.isnull().sum()

gender              0
age                 0
id                  0
became_member_on    0
income              0
joined_in_year      0
dtype: int64